In [11]:
# Librerias
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dash
from dash import dcc
from dash import html
import pandas as pd
import plotly.express as px
import plotly.express as px
import geopandas as gpd

In [27]:
# Lectura de la base de datos
archivo_pickle = 'Entrenamiento_cronicas_N.pkl'
with open(archivo_pickle, 'rb') as file:
    data_ori = pickle.load(file)
data_ori

,EPS,NUM_IDE,FECHA_NACI,SEXO,DPTO,MUNI,DIAS_COMP,V8,id,Valor_Neto_Chunk,...,ARTROSIS,RENAL_OTRA,RENAL_RENALDELARGADURACION,TRANSPLANTE,RENAL_INSUFICIENCIARENALOTRA,RENAL_INSUFICIENCIARENALCRONICA,ANOMALIASGENETICASYCONGENITAS,CANCER_TERAPIACANCER,Grupo_Edad,Zona
rownames,,,,,,,,,,,,,,,,,,,,,
1184873,EPS002,7B7119A8-7D83-4DAF-ABC9-D33642AF26FD,1999-05-14 00:00:00.000,F,50,1,360,B,EPS002-7B7119A8-7D83-4DAF-ABC9-D33642AF26FD,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,N
1587467,EPS013,D36643BD-6CCC-4608-94D8-6B167B226CB5,1988-08-12 00:00:00.000,M,76,1,336,C,EPS013-D36643BD-6CCC-4608-94D8-6B167B226CB5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,C
753062,EPS017,919EBF57-FA41-4DD2-AF8A-CB237B47685E,1996-03-15 00:00:00.000,F,11,1,333,B,EPS017-919EBF57-FA41-4DD2-AF8A-CB237B47685E,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,C
1802563,EPS002,18A7B56B-23F0-4319-BA50-7A497F3FCF18,1998-04-23 00:00:00.000,M,76,1,84,B,EPS002-18A7B56B-23F0-4319-BA50-7A497F3FCF18,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,C
1190692,EPS037,C5BD60C4-B826-4706-9610-E01F83A7C6AB,1990-10-20 00:00:00.000,F,15,776,180,B,EPS037-C5BD60C4-B826-4706-9610-E01F83A7C6AB,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14721902,EPS005,6A239F21-AE92-4681-BC47-F070C899B960,1980-08-17 00:00:00.000,M,88,1,360,C,EPS005-6A239F21-AE92-4681-BC47-F070C899B960,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,Z
9275642,EPS037,E40491D5-6630-4C46-B927-CF3AEFBA9FD5,1964-04-23 00:00:00.000,F,76,111,215,B,EPS037-E40491D5-6630-4C46-B927-CF3AEFBA9FD5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,N
600067,EPS002,0AC1F68B-BB7D-4F45-96B1-C2F11B8CD578,1981-08-18 00:00:00.000,M,11,1,30,B,EPS002-0AC1F68B-BB7D-4F45-96B1-C2F11B8CD578,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,C


In [6]:
import pickle
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Cargar el modelo desde el archivo pickle
with open('model_sinpriv.pkl', 'rb') as file:
    loaded_model = pickle.load(file)
df = data_ori    
# Eliminar columnas donde todas las entradas son cero
df = df.loc[:, (df != 0).any(axis=0)]
df['DIAS_COMP'] = df['DIAS_COMP'].astype(float)
df.drop(columns='NUM_IDE', inplace=True)
df.drop(columns='FECHA_NACI', inplace=True)
df.drop(columns='id', inplace=True)
df.drop(columns='MUNI', inplace=True)

# Eliminar filas con valores nulos
df = df.dropna()

# Binarizar las variables binarias especificadas
binary_columns = [
    'SIDA_VIH', 'TUBERCULOSIS', 'CANCER_OTROSCANCER', 'CANCER_ORGANOSDIGESTIVOS',
    'CANCER_ORGANOSRESPIRATORIOS', 'CANCER_MELANOMAYDELAPIEL', 'CANCER_MAMA',
    'CANCER_OTROSGENITALESFEMENINOS', 'CANCER_CERVIXINVASIVO', 'CANCER_GENITALESMASCULINOS',
    'CANCER_TEJIDOLINFATICOYAFINES', 'CANCER_CERVIXINSITU', 'AUTOINMUNE', 'DIABETES',
    'SINDROMESCONVULSIVOS(EPILEPSIA)', 'ENFERMEDADCARDIOVASCULAR_OTRA', 'ENFERMEDADCARDIOVASCULAR_HIPERTENSION',
    'ENFERMEDADPULMONARDELARGADURACION', 'ASMA', 'ARTRITISPIOGENASYREACTIVAS',
    'ARTRITIS', 'ARTROSIS', 'RENAL_OTRA', 'RENAL_RENALDELARGADURACION', 'TRANSPLANTE',
    'RENAL_INSUFICIENCIARENALOTRA', 'RENAL_INSUFICIENCIARENALCRONICA', 'ANOMALIASGENETICASYCONGENITAS',
    'CANCER_TERAPIACANCER'
]
df[binary_columns] = df[binary_columns].fillna(0).astype(int)
df = df[df[binary_columns].sum(axis=1) > 0]

# Aplicar One-Hot Encoding a las variables categóricas
categorical_columns = ['EPS', 'SEXO', 'DPTO', 'V8', 'Grupo_Edad', 'Zona']
encoder = OneHotEncoder()
encoded_categoricals = encoder.fit_transform(df[categorical_columns])
encoded_df = pd.DataFrame(encoded_categoricals.toarray(), columns=encoder.get_feature_names_out(categorical_columns), index=df.index)

# Concatenar los datos codificados con el resto del dataframe
df = pd.concat([df.drop(categorical_columns, axis=1), encoded_df], axis=1)
df

C:\Users\ASUS\AppData\Local\Temp\ipykernel_20536\633739475.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DIAS_COMP'] = df['DIAS_COMP'].astype(float)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_20536\633739475.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns='NUM_IDE', inplace=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_20536\633739475.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

,DIAS_COMP,Valor_Neto,DIAS_ESTAN_Neto,Edad,SIDA_VIH,TUBERCULOSIS,CANCER_OTROSCANCER,CANCER_ORGANOSDIGESTIVOS,CANCER_ORGANOSRESPIRATORIOS,CANCER_MELANOMAYDELAPIEL,...,Grupo_Edad_3,Grupo_Edad_4,Grupo_Edad_5,Grupo_Edad_6,Grupo_Edad_7,Grupo_Edad_8,Grupo_Edad_9,Zona_C,Zona_N,Zona_Z
rownames,,,,,,,,,,,,,,,,,,,,,
1587467,336.0,312012.0,277.0,23.0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1649659,360.0,319699.0,130.0,4.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
29701,360.0,1084832.0,2495.0,53.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
258375,283.0,544316.0,289.0,58.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1079140,360.0,695539.0,135.0,50.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1343281,360.0,664648.0,394.0,53.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1075476,360.0,236499.0,1.0,26.0,0,0,1,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6479131,360.0,249841.0,195.0,41.0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
